In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.core import *
from local.data.external import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp text.core

# Text core

> Basic function to preprocess text before assembling it in a `DataBunch`.

In [ ]:
#export 
from multiprocessing import Process, Queue
import spacy,html
from spacy.symbols import ORTH

## Preprocessing rules

The following are rules applied to texts before or after it's tokenized.

In [ ]:
#export
#special tokens
UNK, PAD, BOS, EOS, FLD, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxfld xxrep xxwrep xxup xxmaj".split()

In [ ]:
#export
_re_spec = re.compile(r'([/#\\])')

def spec_add_spaces(t):
    "Add spaces around / and #"
    return _re_spec.sub(r' \1 ', t)

In [ ]:
test_eq(spec_add_spaces('#fastai'), ' # fastai')
test_eq(spec_add_spaces('/fastai'), ' / fastai')
test_eq(spec_add_spaces('\\fastai'), ' \\ fastai')

In [ ]:
#export
_re_space = re.compile(' {2,}')

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return _re_space.sub(' ', t)

In [ ]:
test_eq(rm_useless_spaces('a  b   c'), 'a b c')

In [ ]:
#export
_re_rep = re.compile(r'(\S)(\1{3,})')

def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    return _re_rep.sub(_replace_rep, t)

It starts replacing at 4 repetitions of the same character or more.

In [ ]:
test_eq(replace_rep('aaa'), 'aaa')
test_eq(replace_rep('aaaa'), f' {TK_REP} 4 a ')

In [ ]:
#export
_re_wrep = re.compile(r'(?:\s|^)(\w+)\s+((?:\1\s+){2,})\1(\s|\W|$)')

#hide

Matches any word repeated at least four times with spaces between them
```
(?:\s|^)          Non-catching group with either a whitespace character or the beginning of text
(\w+)             Catching group of any alphanumeric character
\s+               One or more whitespace
((?:\1\s+){2,})   Catching group of a repetition of two or more times \1 followed by one or more whitespace
\1                Occurence of \1
(\s|\W|$)         Catching group of last whitespace, non alphanumeric character or end of text
```

In [ ]:
#export
def replace_wrep(t):
    "Replace word repetitions: word word word word -> TK_WREP 4 word"
    def _replace_wrep(m):
        c,cc,e = m.groups()
        return f' {TK_WREP} {len(cc.split())+2} {c} {e}'
    return _re_wrep.sub(_replace_wrep, t)

It starts replacing at 4 repetitions of the same word or more.

In [ ]:
test_eq(replace_wrep('ah ah'), 'ah ah')
test_eq(replace_wrep('ah ah ah ah'), f' {TK_WREP} 4 ah ')
test_eq(replace_wrep('ah ah   ah  ah'), f' {TK_WREP} 4 ah ')
test_eq(replace_wrep('ah ah ah ah '), f' {TK_WREP} 4 ah  ')
test_eq(replace_wrep('ah ah ah ah.'), f' {TK_WREP} 4 ah .')
test_eq(replace_wrep('ah ah ah ahi'), f'ah ah ah ahi')

In [ ]:
#export
def fix_html(x):
    "Various messy things we've seen in documents"
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace('nbsp;', ' ').replace(
        '#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace('<br />', "\n").replace(
        '\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(' @-@ ','-')
    return html.unescape(x)

In [ ]:
test_eq(fix_html('#39;bli#146;'), "'bli'")
test_eq(fix_html('Sarah amp; Duck'), 'Sarah & Duck')
test_eq(fix_html('a nbsp; #36;'), 'a   $')
test_eq(fix_html('\\" <unk>'), f'" {UNK}')
test_eq(fix_html('quot;  @.@  @-@ '), "' .-")
test_eq(fix_html('<br />text\\n'), '\ntext\n')

In [ ]:
#export
_re_all_caps = re.compile(r'(\s|^)([A-Z]+[^a-z\s]*)(?=(\s|$))')

#hide

Catches any word in all caps, even with ' or - inside
```
(\s|^)        Catching group with either a whitespace or the beginning of text
([A-Z]+       Catching group with one capitalized letter or more...
[^a-z\s]*)    ...followed by anything that's non lowercase or whitespace
(?=(\s|$))    Look ahead for a space of end of text
```
The look ahead is there to not move the pointer ahead of the next space in case we have consecutive words in all caps.

In [ ]:
#export
def replace_all_caps(t):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    def _replace_all_caps(m):
        tok = f'{TK_UP} ' if len(m.groups()[1]) > 1 else ''
        return f"{m.groups()[0]}{tok}{m.groups()[1].lower()}"
    return _re_all_caps.sub(_replace_all_caps, t)

In [ ]:
test_eq(replace_all_caps("I'M SHOUTING"), f"{TK_UP} i'm {TK_UP} shouting")
test_eq(replace_all_caps("I'm speaking normally"), "I'm speaking normally")
test_eq(replace_all_caps("I am speaking normally"), "i am speaking normally")

In [ ]:
#export
_re_maj = re.compile(r'(\s|^)([A-Z][^A-Z\s]*)(?=(\s|$))')

#hide

Catches any capitalized word
```
(\s|^)       Catching group with either a whitespace or the beginning of text
([A-Z]       Catching group with exactly one capitalized letter...
[^A-Z\s]*)   ...followed by anything that's not uppercase or whitespace
(?=(\s|$))   Look ahead for a space of end of text
```
The look ahead is there to not move the pointer ahead of the next space in case we have consecutive words in all caps.

In [ ]:
#export
def replace_maj(t):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    def _replace_maj(m):
        tok = f'{TK_MAJ} ' if len(m.groups()[1]) > 1 else ''
        return f"{m.groups()[0]}{tok}{m.groups()[1].lower()}"
    return _re_maj.sub(_replace_maj, t)

In [ ]:
test_eq(replace_maj("Jeremy Howard"), f'{TK_MAJ} jeremy {TK_MAJ} howard')
test_eq(replace_maj("I don't think there is any maj here"), ("i don't think there is any maj here"),)

In [ ]:
#export
def lowercase(t, add_bos=True, add_eos=False):
    "Converts `t` to lowercase"
    return (f'{BOS} ' if add_bos else '') + t.lower().strip() + (f' {EOS}' if add_eos else '')

In [ ]:
defaults.text_spec_tok = [UNK, PAD, BOS, EOS, FLD, TK_REP, TK_WREP, TK_UP, TK_MAJ]
defaults.text_proc_rules = [fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces,
                            replace_all_caps, replace_maj, lowercase]
defaults.text_token_sep = '\u2581'

## Tokenizing

A tokenizer is a class that must implement a `pipe` method. This `pipe` method receives a generator of texts and must return a generator with their tokenized versions. Here is the most basic example:

In [ ]:
#export
class BaseTokenizer():
    "Basic tokenizer that just splits on spaces"
    def __init__(self, **kwargs): pass
    def pipe(self, items): 
        for t in items: yield t.split()

In [ ]:
tok = BaseTokenizer()
for t in tok.pipe(["This is a text"]): test_eq(t, ["This", "is", "a", "text"])

In [ ]:
#export
class SpacyTokenizer():
    "Spacy tokenizer for `lang`"
    def __init__(self, lang='en', special_toks=None, batch_size=5000):
        special_toks = ifnone(special_toks, defaults.text_spec_tok)
        self.nlp = spacy.blank(lang, disable=["parser", "tagger", "ner"])
        for w in special_toks: self.nlp.tokenizer.add_special_case(w, [{ORTH: w}])
        self.batch_size=batch_size
    
    def pipe(self, items):
        for doc in self.nlp.pipe(items, batch_size=self.batch_size):
            yield [d.text for d in doc]

In [ ]:
tok = SpacyTokenizer()
for t in tok.pipe(["This isn't the easiest text."]): 
    test_eq(t, ["This", "is", "n't", "the", "easiest", "text", "."])

In [ ]:
#export
def apply_rules(items, rules):
    "Returns a generator that apply `rules`  to `items`"
    for o in items: yield compose(*rules)(o)

In [ ]:
for t in apply_rules(["This is a text"], [replace_maj]): test_eq(t, f"{TK_MAJ} this is a text")

In [ ]:
#export
def tokenize1(text, tok_func=SpacyTokenizer, rules=None, **tok_kwargs):
    "Tokenize one `text` with an instance of `tok_func` and some `rules`"
    rules = L(ifnone(rules, defaults.text_proc_rules))
    tokenizer = tok_func(**tok_kwargs)
    for tok in tokenizer.pipe(apply_rules([text], rules)): return tok

In [ ]:
test_eq(tokenize1("This is a text"), [BOS, TK_MAJ, 'this', 'is', 'a', 'text'])

In [ ]:
#export
def tokenize_items(items, tok_func, rules, output_func, output_queue, data_queue=None, **tok_kwargs):
    'Tokenize `items` with an instance of `tok_func` and some `rules`'
    tokenizer = tok_func(**tok_kwargs)
    if data_queue: counts = Counter()
    for o,tok in zip(items, tokenizer.pipe(apply_rules(items, rules))):
        output_queue.put(output_func(o, tok))
        if data_queue: counts.update(Counter(tok))
    if data_queue: data_queue.put(counts)

The main function that will be called during one of the processes handling tokenization. It will create an instance of a tokenizer with `tok_func` and `tok_kwargs`, then iterate through the `items`, apply them `rules`, tokenize them, then apply `output_func` to the original item and the tokens and put the result in `output_queue`.

If a `data_queue` is passed, we count the different tokens and return the Counter in it at the end, to save the vocab.

In [ ]:
def launch_process(items, tok_func, rules, output_func):#Just for testing
    output_queue = Queue(1)
    process = Process(target=tokenize_items, args=(items, tok_func, rules, output_func, output_queue))
    process.start()
    for _ in range(2): print(output_queue.get())
    process.join()

texts = ["this is a text", "this is another text"]
rules = [lambda i: texts[i]]
output_func = lambda i,t: t
test_stdout(lambda: launch_process([0,1], BaseTokenizer, rules, output_func),
           """['this', 'is', 'a', 'text']
['this', 'is', 'another', 'text']""")

### Tokenize texts in files

In [ ]:
#export
def create_folders(path, output_dir, include=None):
    "Scan `path` and create the same folder architecture in `output_dir`"
    output_dir = Path(output_dir)
    os.makedirs(output_dir, exist_ok=True)
    for i,(p,d,f) in enumerate(os.walk(path)): # returns (dirpath, dirnames, filenames)
        if include is not None and i==0: d[:] = [o for o in d if o in include]
        else:                            d[:] = [o for o in d if not o.startswith('.')]
        for x in d: os.makedirs(output_dir/(Path(p)/Path(x)).relative_to(path), exist_ok=True)

If `include` is specified, only the folders in `include` will be considered.

In [ ]:
path = Path('tmp')
os.makedirs(path, exist_ok=True)
for i,d in enumerate(['a', 'b', 'c']): 
    os.makedirs(path/d, exist_ok=True)
    for k in range(i+1): os.makedirs(path/d/str(k), exist_ok=True)
create_folders(path, 'tmp1')
out = Path('tmp1')
assert out.is_dir()
for i,d in enumerate(['a', 'b', 'c']): 
    assert (out/d).is_dir()
    for k in range(i+1): assert (out/d/str(k)).is_dir()
shutil.rmtree(path)
shutil.rmtree(out)

In [ ]:
#hide
#test with include
path = Path('tmp')
os.makedirs(path, exist_ok=True)
for i,d in enumerate(['a', 'b', 'c']): 
    os.makedirs(path/d, exist_ok=True)
    for k in range(i+1): os.makedirs(path/d/str(k), exist_ok=True)
create_folders(path, 'tmp1', include=['b', 'c'])
out = Path('tmp1')
assert out.is_dir()
for i,d in enumerate(['a', 'b', 'c']): 
    if i==0:
        assert not (out/d).is_dir()
        continue
    assert (out/d).is_dir()
    for k in range(i+1): assert (out/d/str(k)).is_dir()
shutil.rmtree(path)
shutil.rmtree(out)

Preprocessing function for texts in filenames. Tokenized texts will be saved in a similar fashion in a directory suffixed with `_tok` in the parent folder of `path` (override with `output_dir`).

In [ ]:
#export
def read_text(fname):
    "Read the content of `fname`"
    with open(fname, 'r') as f: return f.read()

In [ ]:
#export
def tokenize_folder(path, extensions=None, include=None, output_dir=None, n_workers=defaults.cpus,
                    rules=None, tok_func=SpacyTokenizer, **tok_kwargs):
    "Tokenize text files in `path` in parallel using `n_workers`"
    path = Path(path)
    extensions = ifnone(extensions, ['.txt'])
    fnames = get_files(path, extensions=extensions, recurse=True, include=include)
    output_dir = Path(ifnone(output_dir, path.parent/f'{path.name}_tok'))
    create_folders(path, output_dir, include=include)
    rules = read_text + L(ifnone(rules, defaults.text_proc_rules.copy()))
    
    output_queue,data_queue = Queue(maxsize=n_workers),Queue(maxsize=n_workers)
    def _output(o, tok):
        out = output_dir/o.relative_to(path)
        with open(out, 'w') as f: f.write(defaults.text_token_sep.join(tok))
        with open(out.parent/f'{out.stem}.len', 'w') as f: f.write(str(len(tok)))
        return 1
            
    processes = [Process(target=tokenize_items,
                         args=(batch, tok_func, rules, _output, output_queue),
                         kwargs={'data_queue': data_queue, **tok_kwargs})
                 for i,batch in enumerate(np.array_split(fnames, n_workers))]
    
    for p in processes: p.start()
    counter = Counter()
    for _ in progress_bar(fnames, leave=False): _ = output_queue.get()
    for _ in processes: counter.update(data_queue.get())
    for p in processes: p.join()
    pickle.dump(counter, open(output_dir/'counter.pkl','wb'))

The result will be in `output_dir` (defaults to a folder in the same parent directory as `path`, with `_tok` added to `path.name`) with the same structure as in `path`. Tokenized texts for a given file will be in the file having the same name in `output_dir`. Additionally, a file with a .len suffix contains the number of tokens and the count of all words is stored in `output_dir/counter.pkl`.

`extensions` will default to `['.txt']` and all text files in `path` are treated unless you specify a list of folders in `include`. `tok_func` is instantiated in each process with `tok_kwargs`, and `rules` (that defaults to `defaults.text_proc_rules`) are applied to each text before going in the tokenizer.

In [ ]:
path = Path('tmp')
os.makedirs(path, exist_ok=True)
for d in ['a', 'b', 'c']: 
    os.makedirs(path/d, exist_ok=True)
    for i in range(5):
        with open(path/d/f'text{i}.txt', 'w') as f: f.write(f"This is an example of text {d} {i}")
tokenize_folder(path)
out = Path('tmp_tok')
assert out.is_dir()
for d in ['a', 'b', 'c']: 
    assert (out/d).is_dir()
    for i in range(5):
        assert (out/d/f'text{i}.txt').is_file()
        assert (out/d/f'text{i}.len').is_file()
        test_eq(read_text(out/d/f'text{i}.txt'), defaults.text_token_sep.join([
            BOS, TK_MAJ, 'this', 'is', 'an', 'example', 'of', 'text', d, str(i)
        ]))
        test_eq(read_text(out/d/f'text{i}.len'), '10')
        
shutil.rmtree(path)
shutil.rmtree(out)

### Tookenize texts in a dataframe

In [ ]:
#export
def join_texts(idx, df, mark_fields=False):
    "Join texts in row `idx` of `df`, marking each field with `FLD` if `mark_fields=True`"
    return ' '.join([(f'{FLD} {i} ' if mark_fields else '') + t for i,t in enumerate(df.iloc[int(idx)].values)])

In [ ]:
#export
def tokenize_df(df, text_cols, n_workers=defaults.cpus, rules=None, mark_fields=None, 
                tok_func=SpacyTokenizer, **tok_kwargs):
    "Tokenize texts in `df[text_cols]` in parallel using `n_workers`"
    text_cols = L(text_cols)
    mark_fields = ifnone(mark_fields, len(text_cols) > 1)
    rules = L(ifnone(rules, defaults.text_proc_rules.copy()))
    rules = partial(join_texts, df=df[text_cols], mark_fields=mark_fields) + rules
    
    output_queue,data_queue = Queue(maxsize=n_workers),Queue(maxsize=n_workers)
    def _output(o, tok): return (o,tok)
            
    processes = [Process(target=tokenize_items,
                         args=(batch, tok_func, rules, _output, output_queue),
                         kwargs={'data_queue': data_queue, **tok_kwargs})
                 for i,batch in enumerate(np.array_split(range(len(df)), n_workers))]
    
    for p in processes: p.start()
    lengths,outputs,counter = np.zeros(len(df)),np.zeros(len(df), dtype=np.object),Counter()
    for _ in progress_bar(range(len(df)), leave=False): 
        i,tok = output_queue.get()
        lengths[i],outputs[i] = len(tok), defaults.text_token_sep.join(tok)
    for _ in processes: counter.update(data_queue.get())
    for p in processes: p.join()
    
    other_cols = [c for c in df.columns if c not in text_cols]
    res = df[other_cols].copy()
    res['text'],res['text_lengths'] = outputs,lengths
    return res, counter

This function returns a new dataframe with the same non-text columns, a colum named text that contains the tokenized texts and a column named text_lengths that contains their respective length. It also returns a counter of all words see to quickly build a vocabulary afterward.

`tok_func` is instantiated in each process with `tok_kwargs`, and `rules` (that defaults to `defaults.text_proc_rules`) are applied to each text before going in the tokenizer. If `mark_fields` isn't specified, it defaults to `False` when there is a single text column, `True` when there are several. In that case, the texts in each of those columns are joined with `FLD` markes followed by the number of the field.

In [ ]:
texts = [f"This is an example of text {i}" for i in range(10)]
df = pd.DataFrame({'text': texts, 'label': list(range(10))}, columns=['text', 'label'])
out,cnt = tokenize_df(df, text_cols='text')
test_eq(list(out.columns), ['label', 'text', 'text_lengths'])
test_eq(out['label'].values, df['label'].values)
for i in range(len(df)):
    test_eq(out['text'][i], defaults.text_token_sep.join([
        BOS, TK_MAJ, 'this', 'is', 'an', 'example', 'of', 'text', str(i)
    ]))
    test_eq(out['text_lengths'][i], 9)
    
#With two columns of text, mark_fields defaults to True
df['text1'] = df['text'].values
out,cnt = tokenize_df(df, text_cols=['text', 'text1'])
test_eq(list(out.columns), ['label', 'text', 'text_lengths'])
test_eq(out['label'].values, df['label'].values)
for i in range(len(df)):
    test_eq(out['text'][i], defaults.text_token_sep.join([
        BOS, FLD, '0', TK_MAJ, 'this', 'is', 'an', 'example', 'of', 'text', str(i),
             FLD, '1', TK_MAJ, 'this', 'is', 'an', 'example', 'of', 'text', str(i)
    ]))
    test_eq(out['text_lengths'][i], 21)

In [ ]:
#export
def tokenize_csv(fname, text_cols, outname=None, n_workers=4, rules=None, mark_fields=None, 
                 tok_func=SpacyTokenizer, header='infer', chunksize=None, **tok_kwargs):
    "Tokenize texts in the `text_cols` of the csv `fname` in parallel using `n_workers`"
    df = pd.read_csv(fname, header=header, chunksize=chunksize)
    outname = Path(ifnone(outname, fname.parent/f'{fname.stem}_tok.csv'))
    kwargs = dict(n_workers=n_workers, pre_rules=pre_rules, post_rules=post_rules, 
                  mark_fields=mark_fields, tok_func=tok_func, **tok_kwargs)
    if chunksize is None:
        out,cnt = tok_df(df, text_cols, **kwargs)
        out.to_csv(outname, header=header, index=False)
    else:
        cnt = Counter()
        for i,dfp in enumerate(df):
            out,c = tok_df(dfp, text_cols, **kwargs)
            out.to_csv(outname, header=header if i==0 else None, index=False, mode='w' if i==0 else 'a')
            cnt.update(c)
    pickle.dump(cnt, open(outname.parent/'counter.pkl', 'wb'))

The result will be written in a new csv file in `outname` (defaults to the same as `fname` with the suffix `_tok.csv`) and will have the same header as the original file, the same non-text columns, a text and a text_lengths column as described in `tokenize_df`.

`tok_func` is instantiated in each process with `tok_kwargs`, and `rules` (that defaults to `defaults.text_proc_rules`) are applied to each text before going in the tokenizer. If `mark_fields` isn't specified, it defaults to `False` when there is a single text column, `True` when there are several. In that case, the texts in each of those columns are joined with `FLD` markes followed by the number of the field.

The csv file is opened with `header` and optionally with blocks of `chunksize` at a time. If this argument is passed, each chunk is processed independtly and saved in the output file to save memory usage.